In [ ]:
# carregar dataset rudementar



In [ ]:
# Limpeza e normalização


In [ ]:
# Gerar variáveis espaciais


In [ ]:
# Cruzar camadas (join espacial)


In [ ]:
# Salvar dataset final


In [ ]:
# Entrega esperada: model_inputs.csv